In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 47.77809012  27.56428276 115.15744798 134.75871573 287.28108034]
 [  7.64672375   4.41157139  18.43056493  21.56768237  45.97837742]
 [ 88.76462936  51.21036309 213.94551691 250.3617751  533.7257842 ]
 [ 19.95836002  11.51443847  48.10476517  56.29281031 120.00603652]
 [ 75.66495318  43.65285761 182.37193845 213.41397052 454.95978261]
 [ 18.9161492   10.913163    45.59276988  53.35324135 113.73940996]
 [119.4231777   68.89798713 287.84047959 336.83460377 718.07013258]
 [ 15.23267296  22.71991899  85.716058    74.8724603   59.63978734]
 [  3.42495327   5.10840487  19.27261838  16.83451606  13.40956279]
 [ 12.98400593  19.36597495  73.06254185  63.81969017  50.83568424]
 [  3.2461412    4.84170212  18.26642165  15.95560927  12.70946808]
 [ 20.49356972  30.56668025 115.31974823 100.73110538  80.23753567]
 [  7.61874311   9.23040031  31.06102962  37.36114412  61.2429735 ]
 [ 28.88291825  34.99276635 117.75343596 141.63738759 232.1742275 ]
 [  7.2210114    8.74853304  29.43950801  35.410

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [10]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [11]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': np.float64(861.9885),
  'Latency (at bend)': np.float64(1.326448252),
  'Slope (after bend)': np.float64(0.000782666)},
 'East Palo Alto': {'Flow (at bend)': np.float64(1001.517857),
  'Latency (at bend)': np.float64(2.213126553),
  'Slope (after bend)': np.float64(0.000584484)},
 'Redwood City': {'Flow (at bend)': np.float64(881.1846667),
  'Latency (at bend)': np.float64(4.892192375),
  'Slope (after bend)': np.float64(0.001563724)},
 'Belmont': {'Flow (at bend)': np.float64(1278.948125),
  'Latency (at bend)': np.float64(1.199911179),
  'Slope (after bend)': np.float64(0.001994138)},
 'San Mateo': {'Flow (at bend)': np.float64(1034.092826),
  'Latency (at bend)': np.float64(5.541006284),
  'Slope (after bend)': np.float64(0.002147262)},
 'Burlingame': {'Flow (at bend)': np.float64(845.15),
  'Latency (at bend)': np.float64(1.503111345),
  'Slope (after bend)': np.float64(0.000306601)},
 'Millbrae': {'Flow (at bend)': np.float64(853.1818182),
  'Laten

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [12]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [13]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [14]:
# def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
#     # Goal: Differentiate between express and general purpose lanes.
    
#     assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
#     assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
#     assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
#     assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
#     coeff_length = 3
    
#     ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
#                             @ np.ones((1, num_edges))
    
#     coeff_full = np.zeros((coeff_length, num_edges, 2))
#     coeff_full[:, :, 0] = coeff_input
#     coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
#     return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum((y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
            x[e, 1, t] += sum(y[(od, g, e, 2, t)] for od in edge_to_od_dict[e] for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in in_indices)
            
#     print()
#     print("in_indices:", in_indices)
#     print("el_indices:", el_indices)
#     print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g, t] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [15]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [16]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [17]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [18]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [19]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [20]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input, no_budget = False):
    
    if no_budget == True:
        B = 0.0
    
#     print("tau (in solve_CBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e] * x[e, 0, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e], 0))
            func += coeff[0, e] * x[e, 1, t] \
                    + 0.5 * coeff[1, e] * num_gp_lanes * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [21]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [22]:
# coeff_input
# demand_array.shape

##  (TODO) Try to initialize tolls and budgets in a smarter way.

In [23]:

T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

num_el = 3
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
# a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array



coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
# VoT_max_el = np.zeros((num_edges, T))
VoT_max = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
#         VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
#                                 for group_index in el_indices])
        VoT_max[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                             for group_index in el_indices + in_indices])
    
#         tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e])
        tau_max_from_latency[e, t] = VoT_max[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \
        
tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e, t] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

print("tau_upper_limit:", tau_upper_limit)
print()
print("B_upper_limit:", B_upper_limit)

d = num_edges * T + 1

tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)

# fraction_tau_max = 0.7
# fraction_B_max_1 = 0.25
# fraction_B_max_2 = 0.5

fraction_tau_max = 0.75
fraction_B_max_1 = 0.5
fraction_B_max_2 = 0.77
assert fraction_B_max_1 < fraction_B_max_2, "We must have fraction_B_max_1 < fraction_B_max_2."

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
# tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)

# TODO: Find smarter way to initialize the new tau (across all lambda coefficient combinations).

fraction_full_tau = 0.25
tau[:, :, 0] = fraction_full_tau * np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)

# B[0] = fraction_B_max * B_upper_limit
# B_upper_limit_init = max([sum(tau[e, t, 0] for e in od_to_edges_list_full[od] for t in range(T) ) \
#                      for od in range(len(od_to_edges_list_full))])
B_upper_limit_init = sum(tau[e, t, 0] for e in range(num_edges) for t in range(T))


# TODO: Find smarter way to initialize the new B (across all lambda coefficient combinations).

B_fraction = 0.6
B[0] = np.random.triangular(0.0, fraction_B_max_1 * B_upper_limit_init, fraction_B_max_2 * B_upper_limit_init) \
            * B_fraction

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print()
print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))
print()
print("B_upper_limit_init:", B_upper_limit_init)
print()
print("B[0]:", B[0])

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[ 6.66325119  6.61038963  6.91467055  6.47634868  6.5380825 ]
 [ 5.20895997  5.16763574  5.40550567  5.06284996  5.11111003]
 [15.         15.         15.         15.         15.        ]
 [15.         15.         15.         15.         15.        ]
 [15.         15.         15.         15.         15.        ]
 [ 3.79800475  3.48940302  3.94131196  3.69147168  3.53565401]
 [ 4.30340013  3.83622369  3.74889532  4.18269084  4.00613873]]

B_upper_limit: 322.6919980500406

tau[:, :, 0]: [[1.10666042 1.10325771 1.62088509 1.54656021 1.14517366]
 [0.3959572  0.87997728 0.77899095 0.89467418 0.90353141]
 [2.77406338 2.54996385 2.28112195 2.80593481 1.14172548]
 [1.86062724 2.28335489 0.50166595 0.73004713 0.41987404]
 [3.4562534  2.41687783 1.00577717 2.15199848 3.31966139]
 [0.49354676 0.54978617 0.81076351 0.53651454 0.42745088]
 [0.48775262 0.64345216 0.69107334 0.97183042 0.86221923]]

np.sum(tau[:, :, 0]): 46.54900471011488

B_upper_limit_init: 46.549004710114886

B[0

In [24]:
# B[0] = B[0] - 2
B[0]

np.float64(9.680558259096795)

## <font color='blue'>Start ignoring here if importing data directly</font> 

In [25]:
# tau[:, :, 0] = argmin_tau

welfare_list = []

# # welfare_list = []


In [26]:
time_1 = time.time()

## Set lambda:

lambda_E, lambda_R, lambda_I = 10.0, 0.0, 1.0

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 1E-5
delta_bar = 0.01

# no_budget = True
no_budget = False

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input, no_budget = no_budget)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input, no_budget = no_budget)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

    time_2 = time.time()

    print()
    print("Iteration count:", i)
    print("Time:", time_2 - time_1)
    print()

time_2 = time.time()
print("Time:", time_2 - time_1)


Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[1.10666042 1.10325771 1.62088509 1.54656021 1.14517366]
 [0.3959572  0.87997728 0.77899095 0.89467418 0.90353141]
 [2.77406338 2.54996385 2.28112195 2.80593481 1.14172548]
 [1.86062724 2.28335489 0.50166595 0.73004713 0.41987404]
 [3.4562534  2.41687783 1.00577717 2.15199848 3.31966139]
 [0.49354676 0.54978617 0.81076351 0.53651454 0.42745088]
 [0.48775262 0.64345216 0.69107334 0.97183042 0.86221923]]
B[i], before projection:
 9.680558259096795
tau_perturbed[:, :, i], before projection:
 [[1.10660199 1.10321088 1.62088661 1.54636603 1.14518257]
 [0.39625632 0.88003866 0.77896247 0.89503581 0.90333792]
 [2.7744483  2.54989374 2.28108637 2.80624005 1.14159913]
 [1.86091699 2.28323578 0.50165596 0.72957126 0.41961835]
 [3.45612059 2.41734163 1.00549318 2.15153864 3.32004228]
 [0.49362886 0.54984955 0.81051228 0.53658591 0.42750905]
 [0.48787819 0.64360953 0.69088286 0.97228431 0.86181015]]
B_perturbed[i], before projection:
 9.68134317965

/opt/anaconda3/lib/python3.13/site-packages/cvxpy/problems/problem.py:167: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "



prob.status: optimal

prob.status: optimal
welfare: 1275283.4050273607
obj_E: 50953.12237139257
obj_R: 8661.277674153376
obj_I: 765752.1813134348

welfare_perturbed: 1275279.4267769083
obj_E_perturbed: 50952.83774652489
obj_R_perturbed: 8659.864344748561
obj_I_perturbed: 765751.0493116595

tau[:, :, i+1], before projection:
 [[1.10582354 1.102587   1.62090688 1.54377921 1.14530117]
 [0.4002411  0.88085626 0.77858312 0.89985337 0.90076033]
 [2.77957614 2.54895966 2.28061244 2.81030649 1.13991601]
 [1.86477695 2.2816491  0.50152296 0.72323183 0.41621213]
 [3.45435134 2.42352022 1.00170998 2.14541279 3.32511632]
 [0.49472257 0.55069385 0.80716537 0.53753679 0.42828407]
 [0.48955107 0.6457061  0.68834544 0.97833086 0.85636054]]
B[i+1], before projection:
 9.69179965709021

B: 9.69179965709021
B_max_wrt_tau_feas: 46.55226300386064
B_feas: 9.69179965709021

tau[:, :, i+1], after projection:
 [[1.1058 1.1026 1.6209 1.5438 1.1453]
 [0.4002 0.8809 0.7786 0.8999 0.9008]
 [2.7796 2.549  2.2806 2


prob.status: optimal

prob.status: optimal
welfare: 494273.3986359961
obj_E: 50638.850607570275
obj_R: 2423.0214879413197
obj_I: 766029.5659800889

welfare_perturbed: 494269.7190066701
obj_E_perturbed: 50638.726255569214
obj_R_perturbed: 2423.508709804402
obj_I_perturbed: 766028.1157491237

tau[:, :, i+1], before projection:
 [[0.47408587 1.07380082 1.1224746  1.47861089 1.33596339]
 [0.6007749  0.96276937 0.45102319 0.48809492 0.77453768]
 [2.09265918 1.67395392 2.72571551 3.1978678  1.1914929 ]
 [1.89740045 2.04089011 3.22326507 1.67832123 1.93521498]
 [1.77921497 2.31396712 1.32307092 2.44602132 1.68154467]
 [0.50692399 0.45639445 0.71489057 0.111036   0.20824941]
 [0.33514205 0.21605742 0.26843419 0.99194547 0.33822695]]
B[i+1], before projection:
 11.430403829612981

B: 11.430403829612981
B_max_wrt_tau_feas: 44.11003629829158
B_feas: 11.430403829612981

tau[:, :, i+1], after projection:
 [[0.4741 1.0738 1.1225 1.4786 1.336 ]
 [0.6008 0.9628 0.451  0.4881 0.7745]
 [2.0927 1.674  2

In [27]:
# dict_latency_params["Belmont"]

In [28]:
min(welfare_list)

np.float64(1275105.1869791055)

In [29]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

# print("argmin_B:", argmin_B)
# print("B[argmin_welfare_list]:", B[argmin_welfare_list])
# print("min_welfare:", min_welfare)
# print()
# print(welfare_list)

# B
print()
print(min_welfare)
print()
print(argmin_welfare_list)
print()
print("len(welfare_list):\n", len(welfare_list))
print()
print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)

# welfare_list[argmin_welfare_list]
# B[argmin_welfare_list]


1275105.1869791055

7

len(welfare_list):
 8

argmin_tau:
 [[1.102  1.1004 1.622  1.5412 1.15  ]
 [0.3992 0.8826 0.7729 0.8947 0.8973]
 [2.7742 2.5431 2.2842 2.8069 1.1315]
 [1.8603 2.2761 0.5023 0.7231 0.4215]
 [3.4454 2.4147 1.0009 2.1443 3.3253]
 [0.497  0.5437 0.8073 0.5375 0.4275]
 [0.4829 0.6493 0.6878 0.9809 0.8571]]

argmin_B:
 9.6924


In [30]:
# TODO: find "argmin_y" from the above code.

argmin_y = y_values
argmin_y

{(0, 0, 0, 0, 0): np.float64(47.77801984766525),
 (0, 0, 0, 1, 0): np.float64(1.037523270988523e-06),
 (0, 0, 0, 2, 0): np.float64(6.893321579615749e-05),
 (0, 1, 0, 0, 0): np.float64(27.56421077858137),
 (0, 1, 0, 1, 0): np.float64(3.3208948144215334e-06),
 (0, 1, 0, 2, 0): np.float64(6.848806475483025e-05),
 (0, 2, 0, 0, 0): np.float64(115.15736998698165),
 (0, 2, 0, 1, 0): np.float64(7.800823845586975e-06),
 (0, 2, 0, 2, 0): np.float64(6.94668099978335e-05),
 (0, 3, 0, 0, 0): np.float64(2.6332957316592834e-05),
 (0, 3, 0, 1, 0): np.float64(134.75868939302825),
 (0, 4, 0, 0, 0): np.float64(0.0002930983131331995),
 (0, 4, 0, 1, 0): np.float64(287.2807872448106),
 (0, 0, 0, 0, 1): np.float64(47.778020289466674),
 (0, 0, 0, 1, 1): np.float64(1.0390411399877054e-06),
 (0, 0, 0, 2, 1): np.float64(6.848989649457393e-05),
 (0, 1, 0, 0, 1): np.float64(27.564211209487055),
 (0, 1, 0, 1, 1): np.float64(3.32572507246742e-06),
 (0, 1, 0, 2, 1): np.float64(6.805232881173002e-05),
 (0, 2, 0, 0, 1)

## <font color='blue'>Finish ignoring here, if importing data directly</font> 

In [31]:
# index_next_init = argmin_welfare_list
# B_next_init = B[index_next_init]
# tau_next_init = tau[:, :, index_next_init]

# print("index_next_init:", index_next_init)
# print("B_next_init:", B_next_init)
# print("tau_next_init:", tau_next_init)


In [32]:
# (OLD) lambda combinations to try to simulate:

# (1, 1, 1)
# (1, 1, 5)
# (1, 1, 10)
# (1, 5, 5)
# (1, 2, 1)
# (1, 2, 5)

# (2, 2, 1)
# (2, 2, 5)
# (2, 5, 5)
# (2, 5, 10)



## <font color='red'>Start compiling here, if importing data directly; otherwise ignore.</font> 

In [33]:
# # ## Initialize lambda:

# # lambda_E, lambda_R, lambda_I = 1.0, 10.0, 1.0

# ## Initialize tau, alpha values:

# if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
#     str_int_lambda_E = str(int(lambda_E))
# else:
#     str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

# if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
#     str_int_lambda_R = str(int(lambda_R))
# else:
#     str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
# if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
#     str_int_lambda_I = str(int(lambda_I))
# else:
#     str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
# filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I


# directory_inits = '../data/old___opt_tolls_subsidies_metrics/'
# df_inits = pd.read_csv(directory_inits + 'opt_CBCP___' + filename_segment + '.csv')

# print("filename_segment:", filename_segment)
# print()

# inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
# inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

# argmin_tau = np.zeros((num_edges, T))
# argmin_B = 0

# for e in range(num_edges):
#     for t in range(T):
#         argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
#         argmin_B = inits_B_arr_as_object

# print("argmin_tau:\n", argmin_tau)
# print()
# print("argmin_B:\n", argmin_B)


In [34]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

In [35]:
# argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                     argmin_tau, argmin_B, od_to_edges_array, \
#                                     demand_array_temp, VoT_array, num_el, coeff_input)

# print("argmin_tau:", argmin_tau)
# print("argmin_B:", argmin_B)
# # print("min_welfare:", min_welfare)
# print()
# print("argmin_y:\n")

# argmin_y


## <font color='red'>Finish compiling here, if importing data directly; otherwise ignore.</font> 

# Store argmin y into csv for processing elsewhere.

In [36]:
## <font color='red'>Code edits start here (June 20) </font> 

In [37]:

# Compress k = 0 and k = 1 flows for eligible user groups.
argmin_y_to_save_dict = {}

for key, value in argmin_y.items():
    assert len(list(key)) == 5, "We must have len(list(key)) == 5."
    
    od, g, e, k, t = key
    if g in el_indices:
        if k == 0:
            argmin_y_to_save_dict[(od, g, e, 0, t)] = argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)]
        elif k == 1:
            # Do nothing
            assert 0 == 0
        else:
            assert k == 2, "We should have k == 2."
            argmin_y_to_save_dict[(od, g, e, 1, t)] = argmin_y[(od, g, e, 2, t)]
    else:
        argmin_y_to_save_dict[key] = value
        
# Double check the above code:

for key, value in argmin_y_to_save_dict.items():
    od, g, e, k, t = key
    assert k in [0, 1], "k must be 0 or 1."
    
    if g in el_indices:
        if k == 0:
            argmin_y_to_save_dict[key] == argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)]
        elif k == 1:
            argmin_y_to_save_dict[key] == argmin_y[(od, g, e, 2, t)]
        else:
            assert 0 == 1, "This case should not occur. All k indices should be 0 or 1."
    elif g in in_indices:
        assert argmin_y_to_save_dict[key] == argmin_y[key], \
            "We should have argmin_y_to_save_dict[key] = argmin_y[key]."
    else:
        assert 0 == 1, "This case should not occur. All g indices should be in el_indices or in_indices."


In [38]:
# Change argmin_y format from dict to array:

argmin_y_to_save_array = np.zeros((len(list(argmin_y_to_save_dict.keys())) , 6))

for row_index, key in enumerate(list(argmin_y_to_save_dict.keys())):
#     print(key)
    
    argmin_y_to_save_array[row_index, :5] = np.array(key)
    argmin_y_to_save_array[row_index, -1] = argmin_y_to_save_dict[key]
#     print("row_index, key", (row_index, key))



In [39]:
# Filename ordering for lambdas: (E, R, I)

directory_path = "../data/opt_CBCP_values___" + str(num_el) + "_el_groups/"

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + "___y_CBCP.csv"


column_names = ["od", "g", "e", "k", "t", "flows (y)"]

df_argmin_y_to_save = pd.DataFrame(argmin_y_to_save_array, columns = column_names)
df_argmin_y_to_save["od"] = df_argmin_y_to_save["od"].astype(int)
df_argmin_y_to_save["g"] = df_argmin_y_to_save["g"].astype(int)
df_argmin_y_to_save["e"] = df_argmin_y_to_save["e"].astype(int)
df_argmin_y_to_save["k"] = df_argmin_y_to_save["k"].astype(int)
df_argmin_y_to_save["t"] = df_argmin_y_to_save["t"].astype(int)

df_argmin_y_to_save.to_csv(directory_path + filename, index=False)

df_argmin_y_to_save

,od,g,e,k,t,flows (y)
0,0,0,0,0,0,47.778021
1,0,0,0,1,0,0.000069
2,0,1,0,0,0,27.564214
3,0,1,0,1,0,0.000068
4,0,2,0,0,0,115.157378
...,...,...,...,...,...,...
3995,23,2,6,1,4,0.000090
3996,23,3,6,0,4,0.000042
3997,23,3,6,1,4,343.778094
3998,23,4,6,0,4,0.000447


In [40]:
# demand_array_temp

# tau_upper_limit

# B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

# Compute Cost Metrics

In [41]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice


# argmin_y_in_el_total
# argmin_x




In [42]:
# argmin_y

In [43]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])

# argmin_y


argmin_tau[5, 3]: 0.5375
argmin_x[(5, 0, 3)]: 1040.7556581467327
argmin_x[(5, 1, 3)]: 5604.315051249983


In [44]:
# # num_edges
# num_gp_lanes
# demand_array_temp

In [45]:
# lambda_R = 1

travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express

print("obj_E:\n", obj_E)
print("obj_R:\n", obj_R)
print("obj_I:\n", obj_I)

# np.sum(obj)

obj_E:
 [ 2518.17778484  4313.09240314 13202.35211466  3218.59819893
 18092.35152383  3633.52800384  5966.15291345]
obj_R:
 [  74.6038901   410.20474206 3466.08735658 1392.99463978 2587.90953164
  448.6863308   246.46835203]
obj_I:
 [ 44532.97002994  60305.51020792 196485.99242958  51408.33611485
 273238.97721542  53466.37815027  86224.49340422]


In [46]:
# travel_times
# avg_travel_time
# percent_on_express
# obj_E
# obj_I
# obj_R
# obj
# sum(obj)


In [47]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [48]:
# travel_times
argmin_x

{(0, 0, 0): np.float64(284.4275761345522),
 (0, 1, 0): np.float64(4563.676046806876),
 (0, 0, 1): np.float64(271.52964567156613),
 (0, 1, 1): np.float64(4576.573977269863),
 (0, 0, 2): np.float64(208.40665456616847),
 (0, 1, 2): np.float64(4639.6969683752595),
 (0, 0, 3): np.float64(220.98830873790098),
 (0, 1, 3): np.float64(4627.115314203526),
 (0, 0, 4): np.float64(221.0376009668875),
 (0, 1, 4): np.float64(4627.0660219745405),
 (1, 0, 0): np.float64(1051.8106194165894),
 (1, 1, 0): np.float64(4122.420216612333),
 (1, 0, 1): np.float64(33.53461225419845),
 (1, 1, 1): np.float64(5140.696223774725),
 (1, 0, 2): np.float64(297.4427162443985),
 (1, 1, 2): np.float64(4876.7881197845245),
 (1, 0, 3): np.float64(30.656495396284413),
 (1, 1, 3): np.float64(5143.574340632639),
 (1, 0, 4): np.float64(30.627139158625862),
 (1, 1, 4): np.float64(5143.603696870297),
 (2, 0, 0): np.float64(1070.4654446367558),
 (2, 1, 0): np.float64(5723.2524709923455),
 (2, 0, 1): np.float64(1122.7692657070224),

In [49]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

## Check if "welfare" and "obj" definitions match:

In [50]:
# def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

In [51]:
welfare_obj(T = T, \
            num_edges = num_edges, \
            num_gp_lanes = num_gp_lanes, \
            lambda_E = lambda_E, \
            lambda_R = lambda_R, \
            lambda_I = lambda_I, \
            tau = argmin_tau, \
            demand_array = demand_array, \
            VoT_array = VoT_array, \
            num_el = num_el, \
            od_to_edges_array = od_to_edges_array, \
            y = argmin_y, \
            coeff_input = coeff_input)[0]


np.float64(1275105.1869791055)

In [52]:
sum(obj[e] for e in range(num_edges))
# obj

np.float64(1275105.1869791057)

## Store into opt data array:

In [53]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [54]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,1.10,1.10,1.62,1.54,1.15,1.30,9.69,4.98,15.10,0.41,1.33,1.85,2518.18,44532.97,74.60,69714.75
e=2,0.40,0.88,0.77,0.89,0.90,0.77,0.00,5.58,6.41,5.15,2.22,2.58,4313.09,60305.51,410.20,103436.43
e=3,2.77,2.54,2.28,2.81,1.13,2.31,0.00,17.31,39.78,5.82,5.35,6.44,13202.35,196485.99,3466.09,328509.51
e=4,1.86,2.28,0.50,0.72,0.42,1.16,0.00,18.94,49.20,4.10,1.36,1.91,3218.60,51408.34,1392.99,83594.32
e=5,3.45,2.41,1.00,2.14,3.33,2.47,0.00,19.88,53.62,3.13,6.71,7.88,18092.35,273238.98,2587.91,454162.49
e=6,0.50,0.54,0.81,0.54,0.43,0.56,0.00,14.38,36.07,3.73,1.56,1.83,3633.53,53466.38,448.69,89801.66
e=7,0.48,0.65,0.69,0.98,0.86,0.73,0.00,13.67,37.18,2.14,2.43,2.77,5966.15,86224.49,246.47,145886.02


In [55]:
print("Total societal cost:", sum(opt_data_array[:, -1]))

Total societal cost: 1275105.18


In [56]:
# VoT_array.shape
VoT_array[:, :, 0]

array([[0.03197424, 0.10238604, 0.23974452, 0.58918431, 1.73418502],
       [0.03197424, 0.10238604, 0.23877036, 0.53802556, 2.09613598],
       [0.03197424, 0.10238604, 0.32999103, 0.56537829, 1.7719906 ],
       [0.03197424, 0.10238604, 0.22950133, 0.48764251, 2.04229666],
       [0.03197424, 0.10238604, 0.22228977, 0.5589363 , 1.50632339],
       [0.03197424, 0.10238604, 0.30324721, 0.56034884, 1.60790372],
       [0.03197424, 0.10238604, 0.30763306, 0.69550862, 2.13579654],
       [0.03174571, 0.10243663, 0.25137511, 0.48084708, 1.30151322],
       [0.03174571, 0.10243663, 0.29554871, 0.58407037, 1.39385618],
       [0.03174571, 0.10243663, 0.26550284, 0.56483269, 1.2212903 ],
       [0.03174571, 0.10243663, 0.27630457, 0.55601242, 1.37981045],
       [0.03174571, 0.10243663, 0.29496542, 0.49460834, 1.45324786],
       [0.02927761, 0.10810948, 0.20930151, 0.67659014, 1.86116345],
       [0.02927761, 0.10810948, 0.25720302, 0.4916406 , 1.91445733],
       [0.02927761, 0.10810948, 0.

In [57]:
# # e = 0
# # t = 4

# e = 4
# t = 3

# print("travel_times[(e, 0, t)]:\n", travel_times[(e, 0, t)])
# print()
# print("travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):\n", \
#       travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]) )
# print()
# print("travel_times[(e, 1, t)]:\n", travel_times[(e, 1, t)])
# print()
# print("Total ineligible user express lane flow at (e, t):\n", argmin_y_in_el_total[(e, 0, t, "in")])

# VoT_array[:, in_indices, t]
# # demand_array[:, in_indices]

# # print("")

# obj_R

# for e in range(num_edges):
#     for t in range(T):
#         print(sum(argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices) \
#                     + sum(argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices) )

# VoT_array[:, :, 0]

In [58]:
# random_string = ""
# for idx_rand_str in range(10):
#     random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_CBCP_values___" + str(num_el) + "_el_groups/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + '___tau_B_stats_CBCP.csv'

df_opt_save.to_csv(directory_to_save + filename)

In [59]:
# random_string
# test

# <font color='red'>STOP, END here.</font>

# Compute appropriate initializations for alpha:

In [ ]:
# y: (od, g, e, k, t)
# VoT: (od, g, t)

In [ ]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices, :]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices, :]

# # print("VoT_array[:, in_indices, :]:\n", VoT_array[:, in_indices, :])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :, :]:\n", VoT_array[edge_to_od_dict[e], :, :])

# # VoT_array_trunc_in

# demand_array.shape

In [ ]:
# arr_temp = np.stack([np.array([1, 2, 3, 4, 5]), np.array([3, 1, 4, 5, 2]), np.array([2, 5, 1, 4, 3])]).T
# print(arr_temp)
# # arr_temp[0:, :]
# # print(np.sum(arr_temp))

# # arr_temp[[2, 3], [1, 2]]
# arr_temp[np.ix_([2,3], [1,2])]

In [ ]:
VoT_array[:, :, 0]

## Initialize DBCP flows

In [ ]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary



In [ ]:
# dict_VoTs_demands_annotated = {}

# directory_path = "../data/VoTs_demands_sorted___" + str(num_el) + "_el_groups/"

# e, t = 0, 0
# filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
# df_data_in = pd.read_csv(directory_path + filename_in)
# dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()

# dict_VoTs_demands_annotated[(e, t, "in")]

In [ ]:
start_time = time.time()

dict_VoTs_demands_annotated = {}

directory_path = "../data/VoTs_demands_sorted___" + str(num_el) + "_el_groups/"

for e in range(num_edges):
    for t in range(T):
        
        # Ineligible users:
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_data_in = pd.read_csv(directory_path + filename_in)
        dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()
        
        # Eligible users:
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_data_el = pd.read_csv(directory_path + filename_el)
        dict_VoTs_demands_annotated[(e, t, "el")] = df_data_el.to_numpy()

end_time = time.time()
print("Time:", end_time - start_time)

In [ ]:
# dict_VoTs_demands_annotated

In [ ]:
y_init_DBCP = {}
init_alpha = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
        ## Ineligible user flows:
        
        VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
        demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
            = fill_from_bottom(VoTs_demands_annotated_in[:, 1], argmin_y_in_el_total[(e, 0, t, 'in')])
        demands_annotated_in_gp_DBCP \
            = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP
        
        assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
        assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"
        
        for row_index in range(VoTs_demands_annotated_in.shape[0]):
            od, g = VoTs_demands_annotated_in[row_index, 2:]
            od, g = int(od), int(g)
            y_init_DBCP[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
            y_init_DBCP[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]
        
        ## Eligible user flows:
        
        VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
        demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
            = fill_from_bottom(VoTs_demands_annotated_el[:, 1], argmin_y_in_el_total[(e, 0, t, 'el')])
        demands_annotated_el_gp_DBCP \
            = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP
        
        assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
        assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"
        
        for row_index in range(VoTs_demands_annotated_el.shape[0]):
            od, g = VoTs_demands_annotated_el[row_index, 2:]
            od, g = int(od), int(g)
            y_init_DBCP[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
            y_init_DBCP[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]
            
        ## Compute alpha:
        
        VoT_in_boundary = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
        VoT_el_boundary = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
        init_alpha[e, t] = 1 - VoT_el_boundary/VoT_in_boundary
        
        assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
        # argmin_y_in_el_total[(e, 0, t, 'el')]
        
        

In [ ]:
# list(init_DBCP_flows.keys())
# list(argmin_y.keys())



## Check if DBCP initialization outperforms CBCP

## <font color='red'>Code edits start here </font> 

In [ ]:
# list_1 = [1, 2, 3]
# list_2 = [2, 3, 4]

# [x for x in list_1 if x not in list_2]


In [ ]:
edge_to_od_dict

In [ ]:
# init_DBCP_key_list_comp = [(od, g, e, k, t) for e in range(num_edges) for t in range(T) \
#                               for k in [0, 1] for g in (el_indices + in_indices) for od in edge_to_od_dict[e]]

# list(init_DBCP_flows.keys()) == init_DBCP_key_list_comp

# keys_missing_init_DBCP_flows = [key for key in list(init_DBCP_flows.keys()) \
#                                 if key not in init_DBCP_key_list_comp]
# keys_extra_init_DBCP_flows = [key for key in init_DBCP_key_list_comp \
#                                 if key not in list(init_DBCP_flows.keys())]

# print("keys_missing_init_DBCP_flows:", keys_missing_init_DBCP_flows)
# print("\n")
# print("keys_extra_init_DBCP_flows:", keys_extra_init_DBCP_flows)

In [ ]:
y_in_el_total_init_DBCP = {}
x_init_DBCP = {}

for e in range(num_edges):
    for t in range(T):
        for k in [0, 1]:
            y_in_el_total_init_DBCP[(e, k, t, "in")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in in_indices)
            y_in_el_total_init_DBCP[(e, k, t, "el")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in el_indices)
            
            x_init_DBCP[(e, k, t)] = y_in_el_total_init_DBCP[(e, k, t, "in")] \
                                        + y_in_el_total_init_DBCP[(e, k, t, "el")]


In [ ]:
y_init_DBCP

In [ ]:
y_in_el_total_init_DBCP

In [ ]:
x_init_DBCP

In [ ]:
# lambda_R = 1

travel_times_DBCP = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times_DBCP[(e, 0, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times_DBCP[(e, 1, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

# INCOMPLETE: edit below: 
        
edge_demand_DBCP = {}
avg_travel_time_DBCP = {}
percent_on_express_DBCP = {}
obj_E_DBCP = np.zeros(num_edges)
obj_I_DBCP = np.zeros(num_edges)
obj_R_DBCP = np.zeros(num_edges)
obj_DBCP = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time_DBCP[e, 'el'] = 0.0
    avg_travel_time_DBCP[e, 'in'] = 0.0
    avg_travel_time_DBCP[e, 'ex'] = 0.0
    avg_travel_time_DBCP[e, 'gp'] = 0.0
    
    percent_on_express_DBCP[e, 'el'] = 0.0
    percent_on_express_DBCP[e, 'in'] = 0.0
    percent_on_express_DBCP[e, 'all'] = 0.0
    
    obj_E_DBCP[e] = 0.0
    obj_I_DBCP[e] = 0.0
    obj_R_DBCP[e] = 0.0
    obj_DBCP[e] = 0.0
    
    for t in range(T):
        edge_demand_DBCP[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand_DBCP[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))
    percent_on_express_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))
    percent_on_express_DBCP[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] + edge_demand_DBCP[e, t, 'in'] for t in range(T))
    
    avg_travel_time_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))    
    avg_travel_time_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))    
#     avg_travel_time_DBCP[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time_DBCP[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time_DBCP[e, 'ex'] += sum(travel_times_DBCP[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time_DBCP[e, 'gp'] += sum(travel_times_DBCP[(e, 1, t)] for t in range(T)) / T 
    
    obj_E_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t]) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
    
#                     + sum( y_init_DBCP[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \

    obj_I_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t])

    obj_DBCP[e] = lambda_E * obj_E_DBCP[e] - lambda_R * obj_R_DBCP[e] + lambda_I * obj_I_DBCP[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time_DBCP
# percent_on_express_DBCP
# obj_R_DBCP

sum(obj_DBCP)

In [ ]:
# lambda_R

In [ ]:
# arr = pad_dim(np.array([1, 2, 3, 4])).T
# arr[:,2:]

In [ ]:
# # y: (od, g, e, k, t)
# # VoT: (od, g, t)
# # demand_array: (od, g)

# alpha_init = np.zeros((num_edges, T))

# VoT_in_boundary = np.zeros((num_edges, T))
# VoT_el_boundary = np.zeros((num_edges, T))

# ### Edit: Finish creating dict for DBCP boundary flows.

# # keys: (e, t, "ineligible"/"eligible")
# # values: ("g (boundary)", "VoT (boundary)", "min flow in express lane", "max flow in express lane", \
# #          "flow as given by CBCP")
# # i.e.,
# # keys: (e, t, "in"/"el")
# # values: ("VoT", "min_y_ex", "max_y_ex", "y_from_CBCP")


# dict_DBCP_boundary_flows = {}

# for e in range(num_edges):
# #     print("edge_to_od_dict[e]:", edge_to_od_dict[e])
    
#     for t in range(T):
        
#         print()
#         print()
#         print("e:", e)
#         print("t:", t)
        
#         dict_DBCP_boundary_flows[(e, t, "in")] = {}
#         dict_DBCP_boundary_flows[(e, t, "el")] = {}
        
#         ## Ineligible users:
        
#         VoT_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len(in_indices)))
#         demand_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len(in_indices)))
        
# #         print()
#         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
# #         print()
        
# #         for od_index, od_temp in enumerate(edge_to_od_dict[e]):
# #             for g_index, g_temp in enumerate(in_indices):
# #                 VoT_array_trunc_in[od_index, g_index] = VoT_array[od_temp, g_temp, t] 
# #                 demand_array_trunc_in[od_index, g_index] = demand_array[od_temp, g_temp] 
        
# #         VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :, t]
# #         VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
# #         demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
# #         demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]
        
#         VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
#         demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]
        
# #         print()
#         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
#         print("VoT_array_trunc_in:\n", VoT_array_trunc_in)
# #         print()

# #         print("np.sum(demand_array_trunc_in):", np.sum(demand_array_trunc_in))
# #         print("(Same sum computed directly):", sum([demand_array[od, g] for od in edge_to_od_dict[e] \
# #                                                     for g in in_indices]))

#         assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                     for g in in_indices])) <= 1E-3
# #         assert np.sum(demand_array_trunc_in_flattened) == sum([demand_array[od, g] for od in edge_to_od_dict[e] \
# #                                                             for g in in_indices])
        
# #         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
# #         print("demand_array_trunc_in.shape:", demand_array_trunc_in.shape)
# #         print("len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])", \
# #               len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]))
# #         print("demand_array_trunc_in:", demand_array_trunc_in)
# #         print("[demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]:", [demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])

#         VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
#         demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
#         VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
#         VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
#         VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
#         VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
#         for row_index in range(VoT_demand_in_cumul.shape[0]):
#             VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])
        
#         assert np.abs(np.sum(VoT_demand_in[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                             for g in in_indices)) <= 1E-3
#         assert np.abs(np.sum(VoT_demand_in_sorted[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                             for g in in_indices)) <= 1E-3
#         assert np.abs(VoT_demand_in_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                 for g in in_indices)) <= 1E-3

#         print("demand_array_trunc_in:\n", demand_array_trunc_in)
#         print("VoT_array_trunc_in:\n", VoT_array_trunc_in)
#         print("VoT_array_trunc_in_flattened:\n", VoT_array_trunc_in_flattened)
#         print("demand_array_trunc_in_flattened:\n", demand_array_trunc_in_flattened)
#         print("VoT_demand_in:\n", VoT_demand_in)
#         print("VoT_demand_in_sorted:\n", VoT_demand_in_sorted)
#         print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)
        

#         ## Eligible users:

#         # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
#         # Sort by VoT
#         # Create cumulative demand array

# #         VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :, t]
# #         VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
# #         demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
# #         demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]

#         VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
#         demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

#         VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
#         demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
#         VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
#         VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
#         VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
#         VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
#         for row_index in range(VoT_demand_el_cumul.shape[0]):
#             VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
# #         print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
# #         print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
# #                                                 for g in el_indices))
# #         print()
#         assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                 for g in el_indices)) <= 1E-3
        
#         ## Ineligible users:
        
#         # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
#         # Find corresponding VoT index -> Value
        
#         VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
#                                      if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
    
#     ### Edit here, 1, start.
        
#         y_in_gp_boundary = VoT_demand_in_cumul[VoT_in_boundary_index, 1] \
#                             - argmin_y_in_el_total[(e, 0, t, 'in')]
        
#         y_in_boundary_demand = VoT_demand_in_sorted[VoT_in_boundary_index, 1]
        
#         y_in_ex_boundary = y_in_boundary_demand - y_in_gp_boundary
        
#         assert y_in_ex_boundary >= 0.0, "We must have y_in_ex_boundary >= 0.0."
#         assert y_in_ex_boundary <= VoT_demand_in_sorted[VoT_in_boundary_index, 1], \
#             "We must have y_in_ex_boundary <= VoT_demand_in_sorted[VoT_in_boundary_index, 1]."
        
#     ### Edit here, 1, end.
         
#         VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
#         assert VoT_in_boundary[e, t] == VoT_demand_in_sorted[VoT_in_boundary_index, 0], \
#             "We must have VoT_in_boundary[e, t] == VoT_demand_in_sorted[VoT_in_boundary_index, 0]"
        
#         print()
        
#         ## Eligible users:
        
#         # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
#         # Find corresponding VoT index -> Value
        
# #         print("VoT_array_trunc_el:", VoT_array_trunc_el)
        
#         print("Total el user flow at e, t:\n", \
#                  sum(demand_array[(od, g)] for od in edge_to_od_dict[e] for g in el_indices))
#         print("argmin_y_in_el_total[(e, 0, t, 'el')]]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
#         print()
        
#         print("demand_array_trunc_el:\n", demand_array_trunc_el)
#         print("VoT_array_trunc_el:\n", VoT_array_trunc_el)
#         print("VoT_array_trunc_el_flattened:\n", VoT_array_trunc_el_flattened)
#         print("demand_array_trunc_el_flattened:\n", demand_array_trunc_el_flattened)
#         print("VoT_demand_el:\n", VoT_demand_el)
#         print("VoT_demand_el_sorted:\n", VoT_demand_el_sorted)
#         print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
        
# #         print("VoT_demand_el_sorted[:, 1]", VoT_demand_el_sorted[:, 1])
# #         print("VoT_demand_el_cumul[:, 1]", VoT_demand_el_cumul[:, 1])
# #         print("VoT_demand_el_cumul[0, 1]:\n", VoT_demand_el_cumul[0, 1])
#         print("(Same sum computed directly):\n", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                    for g in el_indices))
# #         print()
#         assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
#                         for g in el_indices)) <= 1E-3
        
#         VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
#                                      if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')] - 1E-3])
        
#     ### Edit here, 2, start.
    
#         y_el_gp_boundary = VoT_demand_el_cumul[VoT_el_boundary_index, 1] \
#                             - argmin_y_in_el_total[(e, 0, t, 'el')]
        
#         y_el_boundary_demand = VoT_demand_el_sorted[VoT_el_boundary_index, 1]
        
#         y_el_ex_boundary = y_el_boundary_demand - y_el_gp_boundary
        
#         assert y_el_ex_boundary >= 0.0, "We must have y_el_ex_boundary >= 0.0."
#         assert y_el_ex_boundary <= VoT_demand_el_sorted[VoT_el_boundary_index, 1], \
#             "We must have y_in_ex_boundary <= VoT_demand_el_sorted[VoT_el_boundary_index, 1]."
    
#         y_ex_boundary = y_in_ex_boundary + y_el_ex_boundary
        
#         VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
#         assert VoT_el_boundary[e, t] == VoT_demand_el_sorted[VoT_el_boundary_index, 0], \
#             "We must have VoT_el_boundary[e, t] == VoT_demand_el_sorted[VoT_el_boundary_index, 0]"
        
#         dict_DBCP_boundary_flows[(e, t, "in")]["VoT"] = VoT_in_boundary[e, t]
#         dict_DBCP_boundary_flows[(e, t, "in")]["min_y_ex"] = max(y_ex_boundary - y_el_boundary_demand, 0)
#         dict_DBCP_boundary_flows[(e, t, "in")]["max_y_ex"] = min(y_ex_boundary, y_in_boundary_demand)
#         dict_DBCP_boundary_flows[(e, t, "in")]["y_from_CBCP"] = y_in_ex_boundary
               
#         dict_DBCP_boundary_flows[(e, t, "el")]["VoT"] = VoT_el_boundary[e, t]
#         dict_DBCP_boundary_flows[(e, t, "el")]["min_y_ex"] = max(y_ex_boundary - y_in_boundary_demand, 0)
#         dict_DBCP_boundary_flows[(e, t, "el")]["max_y_ex"] = min(y_ex_boundary, y_el_boundary_demand)
#         dict_DBCP_boundary_flows[(e, t, "el")]["y_from_CBCP"] = y_el_ex_boundary
        
#         assert dict_DBCP_boundary_flows[(e, t, "in")]["min_y_ex"] - 1E-3 \
#             <= dict_DBCP_boundary_flows[(e, t, "in")]["y_from_CBCP"] \
#             <= dict_DBCP_boundary_flows[(e, t, "in")]["max_y_ex"] + 1E-3, \
#             "We must have dict_DBCP_boundary_flows[(e, t, in)][min_y_ex] <= dict_DBCP_boundary_flows[(e, t, in)][max_y_ex] + 1E-3"
#         assert dict_DBCP_boundary_flows[(e, t, "el")]["min_y_ex"] - 1E-3 \
#             <= dict_DBCP_boundary_flows[(e, t, "el")]["y_from_CBCP"] \
#             <= dict_DBCP_boundary_flows[(e, t, "el")]["max_y_ex"] + 1E-3, \
#             "We must have dict_DBCP_boundary_flows[(e, t, el)][min_y_ex] <= dict_DBCP_boundary_flows[(e, t, el)][max_y_ex] + 1E-3"
        
        
#     ### Edit here, 2, end.
    
#         if argmin_y_in_el_total[(e, 0, t, 'el')] <= 1E-3:
#             alpha_init[e, t] = 0
#         else:
#             alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
#         print("VoT_el_boundary_index:", VoT_el_boundary_index)
#         print("VoT_in_boundary_index:", VoT_in_boundary_index)
#         print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
#         print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
#         print("alpha_init[e, t]:", alpha_init[e, t])
        
#         assert 0.0 <= alpha_init[e, t] <= 1.0


In [ ]:
# VoT_demand_in_sorted

In [ ]:
# dict_DBCP_boundary_flows

In [ ]:
# y: (od, g, e, k, t)
# VoT: (od, g, t)
# demand_array: (od, g)



el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

In [ ]:
# argmin_y

In [ ]:
# VoT_array[:, :, 0]

In [ ]:
# DBCP_equiv_argmin_y = {}

# # key_0 = list(argmin_y.keys())[0]
# # od, g, e, k, t = key_0

# for key in argmin_y.keys():    
#     od, g, e, k, t = key
#     if k == 0:
#         if g in [0, 1]:
#             if VoT_array[od, g, t] > dict_DBCP_boundary_flows[(e, t, 'el')]['VoT'] + 1E-3:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g]
#             elif VoT_array[od, g, t] < dict_DBCP_boundary_flows[(e, t, 'el')]['VoT'] - 1E-3:
#                 DBCP_equiv_argmin_y[key] = 0.0
#             else:
#                 DBCP_equiv_argmin_y[key] = dict_DBCP_boundary_flows[(e, t, 'el')]['y_from_CBCP']
#         if g in [2, 3, 4]:
#             if VoT_array[od, g, t] > dict_DBCP_boundary_flows[(e, t, 'in')]['VoT'] + 1E-3:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g]
#             elif VoT_array[od, g, t] < dict_DBCP_boundary_flows[(e, t, 'in')]['VoT'] - 1E-3:
#                 DBCP_equiv_argmin_y[key] = 0.0
#             else:
#                 DBCP_equiv_argmin_y[key] = dict_DBCP_boundary_flows[(e, t, 'in')]['y_from_CBCP']
#     if k == 1:
#         if g in [0, 1]:
#             if VoT_array[od, g, t] > dict_DBCP_boundary_flows[(e, t, 'el')]['VoT'] + 1E-3:
#                 DBCP_equiv_argmin_y[key] = 0.0
#             elif VoT_array[od, g, t] < dict_DBCP_boundary_flows[(e, t, 'el')]['VoT'] - 1E-3:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g]
#             else:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g] - dict_DBCP_boundary_flows[(e, t, 'el')]['y_from_CBCP']
#         if g in [2, 3, 4]:
#             if VoT_array[od, g, t] > dict_DBCP_boundary_flows[(e, t, 'in')]['VoT'] + 1E-3:
#                 DBCP_equiv_argmin_y[key] = 0.0
#             elif VoT_array[od, g, t] < dict_DBCP_boundary_flows[(e, t, 'in')]['VoT'] - 1E-3:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g]
#             else:
#                 DBCP_equiv_argmin_y[key] = demand_array[od, g] - dict_DBCP_boundary_flows[(e, t, 'in')]['y_from_CBCP']

    
#     print("key:", key)
#     print("DBCP_equiv_argmin_y[key]:", DBCP_equiv_argmin_y[key])
#     print("demand_array[od, g]:", demand_array[od, g])
#     print()
    
#     assert DBCP_equiv_argmin_y[key] >= 0.0, "We should have DBCP_equiv_argmin_y[key] >= 0.0."
    
# #     print(key)

In [ ]:
DBCP_equiv_argmin_y

In [ ]:
# INCOMPLETE: next steps:
# 
# (1) Check whether the above flows actualy satisfy Nash stationarity.
# (2) Check whether the above flows generate less societal cost than DBCP flows.
# (3) Try to search through the appropriate range of flows, i.e., from 'min_y_ex' to 'max_y_ex', at intervals
# to see if there are DBCP equilibria that yield strictly lower cost.

In [ ]:
# demand_array.shape
num_el

In [ ]:
# Indexing conventions:

# y: (od, g, e, k, t)
# VoT: (od, g, t)

In [ ]:
# argmin_y

In [ ]:
# Try to find DBCP flow corresponding to optimal CBCP flow.

argmin_y_keys = argmin_y.keys()
for key in argmin_y_keys:
    print(key)
    
    # (1) Find what VoT of (od, g, t) is.
    # (2) Find out what the corresponding VoT boundary is (remember to filter for eligible, i.e., g \in (0, 1), \
    ## vs. ineligible, i.e., g \in (0, 1)).
    # (3) If VoT of (od, g, t) > the corresponding VoT boundary, then: (Express lane: Full demand. GP lane: None).
    # (4) If VoT of (od, g, t) < the corresponding VoT boundary, then: (Express lane: None. GP lane: Full demand).
    # (5) If VoT of (od, g, t) = the corresponding VoT boundary, then: Express lane flow given by dict_DBCP_boundary_flows.

#     print()

In [ ]:
# VoT_demand_el_sorted
# VoT_demand_in_sorted

# VoT_array

In [ ]:
VoT_array.shape
# VoT_array

In [ ]:
# VoT_array_trunc_in_flattened

# print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
# print()
# print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
# print()
# print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
# print()
# print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



In [ ]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

In [ ]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

# random_string = ""
# for idx_rand_str in range(10):
#     random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/old___opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
# # argmin_y_in_el_total[(e, 0, t, 'in')]
# # demand_array_trunc_in_flattened

# for e in range(num_edges):
#     for t in range(T):
#         print("e:", e)
#         print("t:", t)
#         print("argmin_y_in_el_total[(e, 0, t, 'el')]:", argmin_y_in_el_total[(e, 0, t, 'el')])
#         print()

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = list(range(num_el))
# in_indices = list(range(num_el, num_groups))
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups, T))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g, t] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)


In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
